In [1]:
import json
from datetime import datetime

import pandas as pd
import requests

from config import Config

In [2]:
class DistributorApi:
    def __init__(self):
        self.base_url = "http://distributor.rules.api.pro.logitravel.internal"
        self.headers = {
            "Accept": "text/plain",
            "Authorization": f"ApiKey {Config.APIKEY}",
            "Content-Type": "application/json",
        }

        self.metadata = {
            "editState": "edit_state",
            "isObsolete": "obsolete",
            "name": "name",
            "description": "description",
            "lastUser": "updated_by",
            "lastDate": "updated_on",
            "tag": "tag",
            "id": "id",
            "rrg": "range",
            "cli": "credential",
            "ps": "paying_society",
            "cp": "refundable",
            "rat": "rate",
            "prv": "provider",
            "hot": "hotel",
            "dest": "destination",
            "mrk": "market",
            "mel": "meal",
            "cid": "check_in",
            "bod": "booking_date",
            "rel": "max_release",
            "isDynamicCommission": "dynamic_commission",
            "dow": "days_of_week",
            "age": "age",
            "room": "room",
            "non": "num_of_nights",
            "hou": "hours",
            "level_mapping": {"t": "level", "l": "list", "f": "from", "u": "to"},
        }

    def get_stopsale_rules(self):
        endpoint = f"/api/organizations/lgt/agencies/products/hot/level_closes/"

        try:
            r = requests.get(f"{self.base_url}{endpoint}", headers=self.headers)
            r.raise_for_status()

            print("Rules retrieved successfully.")

            return r.json()

        except requests.exceptions.HTTPError as errh:
            print("Http Error:", errh)
        except requests.exceptions.ConnectionError as errc:
            print("Error Connecting:", errc)
        except requests.exceptions.Timeout as errt:
            print("Timeout Error:", errt)
        except requests.exceptions.RequestException as err:
            print("Oops: Something Else", err)

    def stopsale_rules_to_df(self, data):
        # Flatten the "lvl" field into the main object
        data = json.loads(
            json.dumps(data), object_hook=lambda obj: {**obj.pop("lvl", {}), **obj}
        )
        rules = data.get("rules", [])

        for rule in rules:
            # Set default "rrg" if missing
            rrg = rule.get("rrg")
            if rrg is None:
                rule["rrg"] = {"f": 0, "t": 0, "u": 0}

            # Rename keys according to metadata
            for key in list(rule.keys()):
                if key in self.metadata:
                    rule[self.metadata[key]] = rule.pop(key)

            # Handle nested dictionary for level mapping
            level_mapping = self.metadata["level_mapping"]
            for key, value in rule.items():
                if isinstance(value, dict):
                    for k in list(value.keys()):
                        if k in level_mapping:
                            value[level_mapping[k]] = value.pop(k)

        return pd.json_normalize(rules, sep="_")

    def delete_rules(self, rule_ids):
        endpoint = "/api/organizations/lgt/agencies/products/hot/level_closes/"

        if rule_ids:
            try:
                r = requests.get(
                    f"{self.base_url}{endpoint}",
                    headers=self.headers,
                    data=json.dumps(rule_ids),
                )
                r.raise_for_status()

                print("Rules deleted successfully.")

                return r.json()

            except requests.exceptions.HTTPError as errh:
                print("Http Error:", errh)
            except requests.exceptions.ConnectionError as errc:
                print("Error Connecting:", errc)
            except requests.exceptions.Timeout as errt:
                print("Timeout Error:", errt)
            except requests.exceptions.RequestException as err:
                print("Oops: Something Else", err)
        else:
            print("No rule IDs provided.")
            return None

    def get_agencies(self):
        endpoint = "/api/organizations/lgt/agencies/"

        try:
            response = requests.get(f"{self.base_url}{endpoint}", headers=self.headers)

            if response.status_code == 200:
                return response.json()
            elif response.status_code == 404:
                print("Error: Agencies not found.")
                return None
            else:
                print(f"Error: Received unexpected status code {response.status_code}.")
                return None

        except requests.exceptions.RequestException as e:
            print(f"An error occurred: {e}")
            return None

In [3]:
api = DistributorApi()

In [4]:
rules = api.get_stopsale_rules()

Rules retrieved successfully.


In [5]:
for rule in rules.get("rules", []):
    print(rule)
    break

{'editState': 1, 'isObsolete': False, 'name': 'BL- Apartamentos Masaru', 'description': 'CDH-19736', 'lastUser': 'maria.ulloa@traveltino.com', 'lastDate': '08-04-2024 17:47:04', 'tag': 1, 'id': '431443', 'lvl': {'rrg': {'t': 0, 'f': 0, 'u': 0}, 'cli': {'t': 1, 'l': ['11986', '11992', '11993', '33284', '10206', '11771', '11994', '11770', '11995', '33182', '33238', '11996', '33533', '33534', '5226', '21950', '31083', '22003', '30830', '8456', '8550', '5815', '6288', '21217', '11730', '11945']}, 'ps': {'t': 0, 'l': None}, 'cp': 0, 'rat': 0, 'prv': {'t': 1, 'l': ['BCONG']}, 'hot': {'t': 1, 'l': ['806']}, 'dest': {'t': 0, 'l': []}, 'mrk': {'t': 0, 'l': None}, 'mel': {'t': 0, 'l': None}, 'cid': {'t': 0, 'f': None, 'u': None}, 'bod': {'t': 0, 'f': None, 'u': None}, 'rel': 0, 'isDynamicCommission': False, 'dow': {'t': 0, 'l': None}, 'age': 0, 'room': {'t': 0, 'l': None}, 'non': {'t': 0, 'l': None}, 'hou': {'t': 0, 'l': None}}}
